# Dot detection using DAOStarFinder

In [1]:
#custom function
from daostarfinder_dotdetection import *
from util import pil_imread
#enhance figure display
%config InlineBackend.figure_format = 'retina'

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def plot_2d_locs_on_2d_image(df_locs_2d_1, df_locs_2d_2, img_2d,add_trace = True, zmax=1000):
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img_2d,
        width=700,
        height=700,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax
    )
    #-------------------------------------------
    
    #For Plotting 2d dots
    #-------------------------------------------
    fig.add_trace(go.Scattergl(
        x=df_locs_2d_1.x,
        y=df_locs_2d_1.y,
        mode='markers',
        marker_symbol='cross',
        marker=dict(
            #maxdisplayed=1000,
            size=4
            ),
        )
    )
    if add_trace == True:
        fig.add_trace(go.Scattergl(
            x=df_locs_2d_2.x,
            y=df_locs_2d_2.y,
            mode='markers',
            marker_symbol='cross',
            marker=dict(
                #maxdisplayed=1000,
                size=4
                ),
            )
        )
    #-------------------------------------------
    
    fig.show()
    
def plot_3d_locs_on_2d_image(df_tiff_1, df_tiff_2, tiff, channel, 
                             raw_src = None, raw_image = False, add_trace = True, zmax=10):
    
    if raw_image == False:

        #PLot All Z's that had dot detection
        #-------------------------------------------
        for z in range(len(tiff[:,channel-1])):
            if add_trace == False:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1, None, tiff[z, channel-1], zmax=zmax, add_trace=add_trace)
            else:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                df_locs_2d_2= df_tiff_2[(df_tiff_2.z > z-1) & (df_tiff_2.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,df_locs_2d_2, tiff[z, channel-1],add_trace=add_trace, zmax=zmax)
    else:
        #read raw image
        tiff = tf.imread(raw_src)
        if len(tiff.shape) == 3:
            tiff = tiff.reshape(1,tiff.shape[0],tiff.shape[1],tiff.shape[2])
        print("shape =", tiff.shape)
        #plot
        for z in range(len(tiff[:,channel-1])):
            if add_trace == False:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,None, tiff[z, channel-1], zmax=zmax, add_trace=add_trace)
            else:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                df_locs_2d_2= df_tiff_2[(df_tiff_2.z > z-1) & (df_tiff_2.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,df_locs_2d_2, tiff[z, channel-1],add_trace=add_trace, zmax=zmax)

In [ ]:
import time
#start time
start = time.time()
hyb = 0
pos = 0
#image sources
img_src = f"/groups/CaiLab/personal/Lex/raw/052922_4kgene/notebook_pyfiles/pre_processed_images/HybCycle_{hyb}/MMStack_Pos{pos}.ome.tif"
img_raw = f'/groups/CaiLab/personal/Lex/raw/052922_4kgene/notebook_pyfiles/dapi_aligned/fiducial_aligned/HybCycle_{hyb}/MMStack_Pos{pos}.ome.tif'
raw_src=None

#img_src: path to image
#HybCycle: which hybcycle are we looking at
#size_cutoff: number of standard deviation away from mean size area
#threshold: absolute pixel intensity the spot must be greater than
#channel: which channel you want to analyze

dots = dot_detection(img_src, HybCycle=hyb, size_cutoff=4, threshold=0.01,channel=1, swapaxes=False)
print(f"This task took {(time.time() - start)/60} minutes")

In [ ]:
# plot dots on top of image
plot_3d_locs_on_2d_image(dots,None, tiff=None, channel=1, raw_src = img_raw, 
                         raw_image = True, add_trace=False, zmax=5000)

In [ ]:
# plot dots on top of image
tiff = pil_imread(img_src, swapaxes=True)
plot_3d_locs_on_2d_image(dots,None, tiff=tiff, channel=1, raw_src = None, 
                         raw_image = False, add_trace=False, zmax=0.1)

# Quick colocalization check

For colocalization betweenn 1st and last hyb. Use the "hyb_coloc" scripts in colocalization_files. The necessary batch files are there for analyzing multiple channels and pos in parallel.

# Combine channels for all pos if you encoded across channels

In [2]:
from tqdm import tqdm

#define channels used
channel = [1,2,3]
#define total number of pos
pos_tot = 66
#define number of z's
num_z = 5
#for each channel in a specific position, grab the locations file
#then, concatenate those files and write it out
for i in tqdm(range(pos_tot)):
    for z in range(num_z):
        df_list = []
        for c in channel:
            paths = f"/groups/CaiLab/personal/Lex/raw/090222_150genes_acrosschannel_3t3/notebook_pyfiles/dots_detected/Channel_{c}/genes_in_cells/Pos{i}/locations_z_{z}.csv"
            try:
                df = pd.read_csv(paths)
            except:
                continue
            df_list.append(df)
        try:
            df_comb = pd.concat(df_list).reset_index(drop=True)
        except:
            continue
        output= Path(f"/groups/CaiLab/personal/Lex/raw/090222_150genes_acrosschannel_3t3/notebook_pyfiles/dots_detected/Channel_All/Pos{i}")
        output.mkdir(parents=True,exist_ok=True)
        output = output / f"locations_z_{z}.csv"
        df_comb.to_csv(str(output))

100%|██████████| 66/66 [06:32<00:00,  5.95s/it]


In [4]:
df = pd.read_csv("/groups/CaiLab/personal/Lex/raw/090222_150genes_acrosschannel_3t3/notebook_pyfiles/dots_detected/Channel_All/Pos0/locations_z_0.csv")

In [6]:
df

,Unnamed: 0,Unnamed: 0.1,hyb,ch,x,y,z,flux,max intensity,sharpness,symmetry,roundness by gaussian fits,size,cell number
0,0,1486,2,1.0,1268.452859,303.201103,0.0,23.460346,0.269564,0.547878,0.006977,-0.019869,14,8
1,1,1566,2,1.0,1285.973042,323.136565,0.0,14.384139,0.164579,0.549306,-0.094415,-0.021065,14,8
2,2,1669,2,1.0,1299.030639,344.146320,0.0,18.526986,0.210896,0.532253,0.086753,-0.209953,16,8
3,3,1685,2,1.0,1309.503120,349.482247,0.0,10.319906,0.116101,0.533944,0.391776,-0.037150,15,8
4,4,1686,2,1.0,1328.216893,349.450108,0.0,13.140815,0.153772,0.578315,0.061565,0.037498,14,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106181,106181,70392,8,3.0,1922.582438,1958.201028,0.0,1.922988,0.020934,0.402244,0.342905,0.664548,18,23
106182,106182,70417,8,3.0,1874.384146,1972.530027,0.0,5.128435,0.060012,0.522896,0.313271,0.081081,15,23
106183,106183,70428,8,3.0,1887.376576,1978.598855,0.0,7.273272,0.089320,0.370357,0.471081,0.241733,22,23
106184,106184,70467,8,3.0,1885.615677,2000.312307,0.0,4.676620,0.052336,0.442428,0.525163,0.418977,18,23


# Check if there are weird dropouts in a certain hyb for each threshold

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
#collect arrays
channel = 4
comb_dots_list = []
for i in range(11):
    path = f"/groups/CaiLab/personal/Lex/raw/020422_20kdash_3t3/notebook_pyfiles/dots_comb/Channel_{channel}/MMStack_Pos0/Threshold_{i}/Dot_Locations/locations_z_0.csv"
    arr1  = pd.read_csv(path)
    comb_dots_list.append(arr1)

In [ ]:
#counts total number of dots across hybs
final_counts = []
for df in comb_dots_list:
    dots_per_hyb = []
    for i in df["hyb"].unique():
        dots_per_hyb.append(len(df[df["hyb"]==i]))
    final_counts.append(dots_per_hyb)

In [ ]:
#generate subplot canvas
x = 4
fig, ax = plt.subplots(x, x, figsize=(8,10), sharex = True, constrained_layout=True)
fig.text(0.5, 0.2, 'HybCycles', ha='center')
fig.text(-0.03, 0.6, 'Total Counts', va='center', rotation='vertical')
i = 0

#fill subplots
for row in ax:
    for col in row:
        try:
            #generate scatter plot of gene
            col.bar(x=np.arange(1,len(final_counts[i])+1,1) ,height=final_counts[i])
            sns.despine()
            col.set_title(f"Threshold {i}")
        except IndexError:
            col.remove()
        i += 1
plt.show()